# CNN Network

In [218]:
import torch
import torch.nn.functional as F
import os
import numpy as np
from typing import Tuple
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import random
from collections import OrderedDict

print(torch.__version__)
print(torchvision.__version__)

curPath=os.path.abspath('')
dataPath=curPath+"/../data"
BATCH_SIZE=64
EPOCHS=1
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
DEVICE = torch.device("mps") 
print(DEVICE)
print(dataPath)


1.13.1
0.13.1a0
mps
/Users/jiaruiye/Desktop/FDU/专业课程/必修课程/人工智能/Projects/PJ1/Part3/../data


In [219]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    
])

train_set = datasets.MNIST(root=dataPath, train=True, download=True, transform=train_transform)
test_set = datasets.MNIST(root=dataPath, train=False, download=True, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride= 2, padding=3, bias= False)
# model.fc = torch.nn.Sequential(OrderedDict([('fc1', torch.nn.Linear(model.fc.in_features, 10)),
                                            # ('output', torch.nn.Softmax(dim=1))
                                            # ]))
# model.fc=torch.nn.Linear(model.fc.in_features,10)
# torch.nn.init.xavier_uniform_(model.fc.weight)
# https://datawhalechina.github.io/thorough-pytorch/
model.fc = torch.nn.Sequential(OrderedDict([('fc1', torch.nn.Linear(model.fc.in_features, 64)),
                                            ('relu1', torch.nn.ReLU()), 
                                            ('dropout',torch.nn.Dropout(0.5)),
                                            ('fc2', torch.nn.Linear(64, 10)),
                                            # ('output', torch.nn.Softmax(dim=1))
                                            ]))
torch.nn.init.xavier_uniform_(model.fc[0].weight)
torch.nn.init.xavier_uniform_(model.fc[3].weight)


criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
# optimizer = torch.optim.Adam([{'params':model.fc.parameters()}], lr=0.00002)
optimizer = torch.optim.SGD([{'params':model.fc.parameters()}], lr=0.00001)
# StepLR = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

model=model.to(DEVICE)
print(model)
print(len(train_loader))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [220]:
def test(model, device, test_loader):
    model.eval()
    sum_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data).to(DEVICE)
            loss = criterion(output, target)  # calculate loss
            sum_loss += loss.item()  # add loss to running total
            pred = output.argmax(dim=1, keepdim=True)  # get predicted class
            correct += pred.eq(target.view_as(pred)).sum().item()  # count correct predictions

    sum_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        sum_loss, correct, len(test_loader.dataset), accuracy))
    return sum_loss, accuracy
def train(model,device,train_loader,epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(data).to(DEVICE)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        if batch_idx%(len(train_loader)//4)==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


In [221]:
# load the saved model
model.load_state_dict(torch.load('97-5-multi-linear-xavier-dropout.pt',map_location=DEVICE))

<All keys matched successfully>

In [222]:
for epoch in range(EPOCHS):
    train(model, DEVICE, train_loader, epoch)
    print("Testing...")
    # print("Train set:")
    # test(model, DEVICE, train_loader)
    print("Test set:")
    loss,accuracy=test(model, DEVICE, test_loader)
    if accuracy>97.47:
        torch.save(model.state_dict(), '97-5-multi-linear-xavier-dropout.pt')
        break

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.086578
Train Epoch: 0 [14976/60000 (25%)]	Loss: 0.146478
Train Epoch: 0 [29952/60000 (50%)]	Loss: 0.071045
Train Epoch: 0 [44928/60000 (75%)]	Loss: 0.107051
Train Epoch: 0 [59904/60000 (100%)]	Loss: 0.127263
Testing...
Test set:
Average loss: 0.0013, Accuracy: 9730/10000 (97.30%)


In [ ]:
test(model, DEVICE, test_loader)

In [193]:
# save the model
torch.save(model.state_dict(), '97-multi-linear-xavier-dropout.pt')